In [4]:
# ============================================
# Garrett RTE Category Manager Analytics (Mock Nielsen-style)
# Colab + Looker Studio ready
# ============================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

# --- Config ---
SEED = 33
np.random.seed(SEED)

# In Colab, you can save to /content. Later we will optionally save to Google Drive.
DATA_DIR = "/content/data"
OUT_DIR = "/content/outputs"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

CSV_PATH = os.path.join(DATA_DIR, "rte_scanner_mock.csv")

# --------------------------------------------
# 1) Generate synthetic "syndicated-like" scanner data
# --------------------------------------------
def generate_mock_scanner_data():
    weeks = pd.date_range("2024-03-03", periods=52, freq="W-SUN")
    retailers = ["Walmart", "Target", "Kroger", "Walgreens", "Costco", "Meijer"]
    regions = ["Midwest", "Northeast", "South", "West"]
    brands = ["Garrett", "Competitor A", "Competitor B"]

    skus = [
        ("Cheddar 6oz", 5.49),
        ("Caramel 6oz", 5.29),
        ("Mix 10oz", 8.99),
        ("Caramel 10oz", 8.49),
        ("Cheddar 10oz", 8.79),
        ("Variety 12pk", 18.99),
    ]

    retailer_cat_mult = {
        "Walmart": 1.30,
        "Target": 1.10,
        "Kroger": 1.05,
        "Walgreens": 0.75,
        "Costco": 1.25,
        "Meijer": 0.90,
    }

    brand_mult = {"Garrett": 0.95, "Competitor A": 1.10, "Competitor B": 0.85}
    region_mult = {"Midwest": 1.15, "Northeast": 1.00, "South": 0.92, "West": 0.96}

    sku_pop = {
        "Cheddar 6oz": 1.15,
        "Caramel 6oz": 1.05,
        "Mix 10oz": 1.25,
        "Caramel 10oz": 0.95,
        "Cheddar 10oz": 1.00,
        "Variety 12pk": 0.70,
    }

    # Distribution proxy: stores_selling
    # Make Garrett underdistributed at Walmart and Costco to create whitespace opportunities
    base_stores = {
        ("Garrett", "Walmart"): 350,
        ("Garrett", "Target"): 420,
        ("Garrett", "Kroger"): 280,
        ("Garrett", "Walgreens"): 210,
        ("Garrett", "Costco"): 120,
        ("Garrett", "Meijer"): 160,
        ("Competitor A", "Walmart"): 700,
        ("Competitor A", "Target"): 560,
        ("Competitor A", "Kroger"): 610,
        ("Competitor A", "Walgreens"): 420,
        ("Competitor A", "Costco"): 210,
        ("Competitor A", "Meijer"): 300,
        ("Competitor B", "Walmart"): 420,
        ("Competitor B", "Target"): 380,
        ("Competitor B", "Kroger"): 340,
        ("Competitor B", "Walgreens"): 260,
        ("Competitor B", "Costco"): 160,
        ("Competitor B", "Meijer"): 210,
    }

    promo_prob = {"Walmart": 0.22, "Target": 0.28, "Kroger": 0.33, "Walgreens": 0.25, "Costco": 0.12, "Meijer": 0.30}
    feature_prob = {"Walmart": 0.12, "Target": 0.18, "Kroger": 0.20, "Walgreens": 0.10, "Costco": 0.08, "Meijer": 0.16}

    rows = []

    for w in weeks:
        week_index = (w - weeks.min()).days / 7
        season = 1.0 + 0.10 * np.sin(2 * np.pi * week_index / 52) + 0.05 * np.cos(2 * np.pi * week_index / 26)

        for retailer in retailers:
            for region in regions:
                for brand in brands:
                    for sku, base_price in skus:
                        promo_flag = int(np.random.rand() < promo_prob[retailer])
                        feature_display = int(np.random.rand() < feature_prob[retailer])

                        discount = np.random.uniform(0.08, 0.25) if promo_flag else 0.0
                        price = round(base_price * (1 - discount), 2)

                        # stores selling: small noise + gentle growth trend
                        base = base_stores[(brand, retailer)]
                        growth = 1.0 + 0.04 * (week_index / 52)  # ~4% growth across the year
                        stores_selling = int(max(25, np.random.normal(loc=base * growth, scale=base * 0.03)))

                        # Demand model: base demand * multipliers * seasonality * promo/feature lift
                        base_units = 18
                        units = (
                            base_units
                            * retailer_cat_mult[retailer]
                            * brand_mult[brand]
                            * region_mult[region]
                            * sku_pop[sku]
                            * season
                        )

                        # Promo + feature lifts
                        if promo_flag:
                            units *= np.random.uniform(1.18, 1.45)
                        if feature_display:
                            units *= np.random.uniform(1.10, 1.28)

                        # Price sensitivity (simple): higher price slightly reduces units
                        units *= (1.0 - 0.02 * max(0, (price - base_price)))

                        # scale by distribution
                        units *= (stores_selling / 500)

                        # noise and floor
                        units = max(0, np.random.normal(loc=units, scale=max(1.0, units * 0.12)))
                        units = int(round(units))

                        dollars = round(units * price, 2)

                        rows.append({
                            "week_ending": w.date().isoformat(),
                            "retailer": retailer,
                            "region": region,
                            "brand": brand,
                            "sku": sku,
                            "price": price,
                            "units": units,
                            "dollars": dollars,
                            "promo_flag": promo_flag,
                            "feature_display": feature_display,
                            "stores_selling": stores_selling,
                        })

    df = pd.DataFrame(rows)

    # Helpful extra fields
    df["on_promo"] = df["promo_flag"].map({0: "No", 1: "Yes"})
    df["featured"] = df["feature_display"].map({0: "No", 1: "Yes"})

    return df


df = generate_mock_scanner_data()
df.to_csv(CSV_PATH, index=False)
df.head(), df.shape


(  week_ending retailer   region    brand           sku  price  units  dollars  \
 0  2024-03-03  Walmart  Midwest  Garrett   Cheddar 6oz   5.49     19   104.31   
 1  2024-03-03  Walmart  Midwest  Garrett   Caramel 6oz   5.29     24   126.96   
 2  2024-03-03  Walmart  Midwest  Garrett      Mix 10oz   8.99     22   197.78   
 3  2024-03-03  Walmart  Midwest  Garrett  Caramel 10oz   6.54     21   137.34   
 4  2024-03-03  Walmart  Midwest  Garrett  Cheddar 10oz   8.79     19   167.01   
 
    promo_flag  feature_display  stores_selling on_promo featured  
 0           0                0             333       No       No  
 1           0                0             348       No       No  
 2           0                0             347       No       No  
 3           1                0             323      Yes       No  
 4           0                0             339       No       No  ,
 (22464, 13))

In [5]:
# ============================================
# 2) Category Trend + Share
# ============================================
df["week_ending"] = pd.to_datetime(df["week_ending"])

weekly = df.groupby(["week_ending", "brand"], as_index=False)[["dollars", "units"]].sum()
weekly_total = df.groupby(["week_ending"], as_index=False)[["dollars", "units"]].sum().rename(columns={"dollars":"cat_dollars","units":"cat_units"})
weekly = weekly.merge(weekly_total, on="week_ending", how="left")
weekly["share_dollars"] = weekly["dollars"] / weekly["cat_dollars"]

# Plot Garrett share trend
garrett_weekly = weekly[weekly["brand"] == "Garrett"].sort_values("week_ending")

plt.figure()
plt.plot(garrett_weekly["week_ending"], garrett_weekly["share_dollars"])
plt.title("Garrett Dollar Share Trend (Synthetic Scanner Data)")
plt.xlabel("Week Ending")
plt.ylabel("Dollar Share")
plt.xticks(rotation=45)
plt.tight_layout()
share_path = os.path.join(OUT_DIR, "garrett_share_trend.png")
plt.savefig(share_path, dpi=200)
plt.close()

share_path


'/content/outputs/garrett_share_trend.png'

In [6]:
# ============================================
# 3) Distribution Whitespace
# High category dollars, low Garrett stores_selling
# ============================================

# Category dollars by retailer (all brands)
retailer_cat = df.groupby("retailer", as_index=False)["dollars"].sum().rename(columns={"dollars":"category_dollars"})

# Garrett distribution proxy by retailer (avg stores_selling)
garrett_dist = df[df["brand"] == "Garrett"].groupby("retailer", as_index=False)["stores_selling"].mean().rename(columns={"stores_selling":"garrett_avg_stores"})

whitespace = retailer_cat.merge(garrett_dist, on="retailer", how="left")

# Create a whitespace score: big category dollars + low distribution
whitespace["category_dollars_rank"] = whitespace["category_dollars"].rank(ascending=False)
whitespace["dist_rank_low_is_good"] = whitespace["garrett_avg_stores"].rank(ascending=True)

whitespace["whitespace_score"] = (
    (whitespace["category_dollars_rank"].max() - whitespace["category_dollars_rank"] + 1) * 0.6
    + (whitespace["dist_rank_low_is_good"].max() - whitespace["dist_rank_low_is_good"] + 1) * 0.4
)

whitespace = whitespace.sort_values("whitespace_score", ascending=False)

whitespace_path = os.path.join(OUT_DIR, "distribution_whitespace.csv")
whitespace.to_csv(whitespace_path, index=False)

whitespace.head(10), whitespace_path


(    retailer  category_dollars  garrett_avg_stores  category_dollars_rank  \
 5    Walmart         810046.75          356.094551                    1.0   
 1     Kroger         565943.59          284.963141                    3.0   
 4  Walgreens         282188.84          213.702724                    4.0   
 3     Target         638245.72          427.232372                    2.0   
 2     Meijer         257343.87          162.729167                    5.0   
 0     Costco         252592.46          122.019231                    6.0   
 
    dist_rank_low_is_good  whitespace_score  
 5                    5.0               4.4  
 1                    4.0               3.6  
 4                    3.0               3.4  
 3                    6.0               3.4  
 2                    2.0               3.2  
 0                    1.0               3.0  ,
 '/content/outputs/distribution_whitespace.csv')

In [7]:
# ============================================
# 4) Assortment Productivity + Recommendations
# dollars/store and units/store, by Garrett SKU and retailer
# ============================================

g = df[df["brand"] == "Garrett"].copy()
sku_perf = g.groupby(["sku"], as_index=False).agg(
    dollars=("dollars", "sum"),
    units=("units", "sum"),
    avg_price=("price", "mean"),
    avg_stores=("stores_selling", "mean")
)

sku_perf["dollars_per_store"] = sku_perf["dollars"] / sku_perf["avg_stores"]
sku_perf["units_per_store"] = sku_perf["units"] / sku_perf["avg_stores"]

# Simple rules for recs
dps_q = sku_perf["dollars_per_store"].quantile([0.33, 0.66]).to_dict()
ups_q = sku_perf["units_per_store"].quantile([0.33, 0.66]).to_dict()

def recommend(row):
    dps = row["dollars_per_store"]
    ups = row["units_per_store"]
    if dps >= dps_q[0.66] and ups >= ups_q[0.66]:
        return "Expand / Hero SKU"
    if dps <= dps_q[0.33] and ups <= ups_q[0.33]:
        return "Rationalize / Review"
    return "Maintain"

sku_perf["recommendation"] = sku_perf.apply(recommend, axis=1)
sku_perf = sku_perf.sort_values(["recommendation", "dollars_per_store"], ascending=[True, False])

assort_path = os.path.join(OUT_DIR, "assortment_recommendations.csv")
sku_perf.to_csv(assort_path, index=False)

sku_perf, assort_path


(            sku    dollars  units  avg_price  avg_stores  dollars_per_store  \
 4      Mix 10oz  144043.55  16770   8.671146  261.213141         551.440672   
 5  Variety 12pk  169848.44   9452  18.190080  260.717949         651.464315   
 2  Cheddar 10oz  112564.28  13426   8.461546  261.112179         431.095479   
 0  Caramel 10oz  102673.20  12774   8.136050  261.080128         393.263174   
 3   Cheddar 6oz   80655.58  15556   5.240881  261.411058         308.539282   
 1   Caramel 6oz   70703.58  14067   5.078429  261.206731         270.680544   
 
    units_per_store     recommendation  
 4        64.200445  Expand / Hero SKU  
 5        36.253737           Maintain  
 2        51.418513           Maintain  
 0        48.927508           Maintain  
 3        59.507812           Maintain  
 1        53.853896           Maintain  ,
 '/content/outputs/assortment_recommendations.csv')

In [8]:
# ============================================
# 5) Pricing + Promotion Performance (Promo ROI proxy)
# ============================================

promo = g.groupby(["sku", "on_promo"], as_index=False).agg(
    avg_price=("price", "mean"),
    total_units=("units", "sum"),
    total_dollars=("dollars", "sum")
)

# Pivot to compute lift
pivot = promo.pivot_table(index="sku", columns="on_promo", values=["avg_price", "total_units", "total_dollars"])
pivot.columns = [f"{a}_{b}" for a, b in pivot.columns]
pivot = pivot.reset_index()

# If any SKU never went on promo or never sold off promo, fill with small values to avoid division issues
for col in ["total_units_Yes","total_units_No","avg_price_Yes","avg_price_No","total_dollars_Yes","total_dollars_No"]:
    if col in pivot.columns:
        pivot[col] = pivot[col].fillna(0)

pivot["unit_lift_ratio"] = np.where(pivot["total_units_No"] > 0, pivot["total_units_Yes"] / pivot["total_units_No"], np.nan)
pivot["price_delta"] = pivot["avg_price_Yes"] - pivot["avg_price_No"]

promo_roi_path = os.path.join(OUT_DIR, "promo_roi_table.csv")
pivot.to_csv(promo_roi_path, index=False)

pivot.sort_values("unit_lift_ratio", ascending=False).head(10), promo_roi_path


(            sku  avg_price_No  avg_price_Yes  total_dollars_No  \
 3   Cheddar 6oz          5.49       4.591445          56568.96   
 5  Variety 12pk         18.99      15.908827         119883.87   
 0  Caramel 10oz          8.49       7.087683          73209.27   
 1   Caramel 6oz          5.29       4.403960          52196.43   
 2  Cheddar 10oz          8.79       7.346655          84058.77   
 4      Mix 10oz          8.99       7.548225         108796.98   
 
    total_dollars_Yes  total_units_No  total_units_Yes  unit_lift_ratio  \
 3           24086.62         10304.0           5252.0         0.509705   
 5           49964.57          6313.0           3139.0         0.497228   
 0           29463.93          8623.0           4151.0         0.481387   
 1           18507.15          9867.0           4200.0         0.425661   
 2           28505.51          9563.0           3863.0         0.403953   
 4           35246.57         12102.0           4668.0         0.385721   
 
  

In [9]:
# ============================================
# 6) Simple Price Elasticity Proxy (Regression)
# For Garrett overall: units ~ price + promo_flag + feature_display + stores_selling
# ============================================

# Build a simple model
X = g[["price", "promo_flag", "feature_display", "stores_selling"]].copy()
y = g["units"].copy()

model = LinearRegression()
model.fit(X, y)

coef = pd.DataFrame({
    "feature": X.columns,
    "coefficient": model.coef_
})
intercept = model.intercept_

coef_path = os.path.join(OUT_DIR, "elasticity_regression_coefficients.csv")
coef.to_csv(coef_path, index=False)

# Plot price vs units (sample for readability)
sample = g.sample(n=min(4000, len(g)), random_state=SEED)

plt.figure()
plt.scatter(sample["price"], sample["units"], alpha=0.25)
plt.title("Price vs Units (Garrett) - Synthetic Data")
plt.xlabel("Price")
plt.ylabel("Units")
plt.tight_layout()
price_units_path = os.path.join(OUT_DIR, "price_vs_units.png")
plt.savefig(price_units_path, dpi=200)
plt.close()

coef, intercept, coef_path, price_units_path


(           feature  coefficient
 0            price    -0.324898
 1       promo_flag     2.376024
 2  feature_display     1.919323
 3   stores_selling     0.047875,
 np.float64(0.5155777141486677),
 '/content/outputs/elasticity_regression_coefficients.csv',
 '/content/outputs/price_vs_units.png')

In [10]:
from google.colab import drive
drive.mount('/content/drive')

# Choose a folder in your Drive
project_folder = "/content/drive/MyDrive/garrett-rte-category-analysis"
os.makedirs(project_folder, exist_ok=True)
os.makedirs(os.path.join(project_folder, "data"), exist_ok=True)
os.makedirs(os.path.join(project_folder, "outputs"), exist_ok=True)

# Copy files
import shutil
shutil.copy("/content/data/rte_scanner_mock.csv", os.path.join(project_folder, "data", "rte_scanner_mock.csv"))
for f in os.listdir("/content/outputs"):
    shutil.copy(os.path.join("/content/outputs", f), os.path.join(project_folder, "outputs", f))

project_folder


Mounted at /content/drive


'/content/drive/MyDrive/garrett-rte-category-analysis'